# Introduction

Our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The UCI Machine Learning Repository. You can also download the dataset directly from this link. The data collection process is described in more details on this page, where you can also find some of the authors' papers.

Let's start by reading in the dataset. You'll be able to find the solutions to this project at this link or by clicking the key icon at the top right of the interface.

# Exploring the dataset

1. Open the SMSSpamCollection file using the `read_csv()` function from the pandas package.
  - The data points are tab separated, so we'll need to use the `sep='\t'` parameter for our `read_csv()` function.
  - The dataset doesn't have a header row, which means we need to use the `header=None` parameter, otherwise the first row will be wrongly used as the header row.
  - Use the `names=['Label', 'SMS']` parameter to name the columns as Label and SMS.
2. Explore the dataset a little.
  - Find how many rows and columns it has.
  - Find what percentage of the messages is spam and what percentage is ham ("ham" means non-spam).

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("SMSSpamCollection.csv", sep="\t", header=None, names=["Label","SMS"])

In [2]:
df.shape

(5572, 2)

In [3]:
df.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [4]:
df.Label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## HAM: 86.59%  -  SPAM: 13.40%

# Training and Test set

On the previous screen, we read in the dataset and saw that about 87% of the messages are ham ("ham" means non-spam), and the remaining 13% are spam. Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The **training set** will have 4,458 messages (about 80% of the dataset).
- The **test set** will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We'll come back to testing toward the end of this guided project, but for now, let's create a training and a test set. We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset. 

## Tasks
1. Start by randomizing the entire dataset by using the `DataFrame.sample()` method.
  - Use the `frac=1` parameter to randomize the entire dataset.
  - Use the `random_state=1` parameter to make sure your results are reproducible.
2. Split the randomized dataset into a training and a test set. The training set should account for 80% of the dataset, and the remaining 20% of the data should be the test set.
3. Find the percentage of spam and ham in both the training and the test set. Are the percentages similar to what we have in the full dataset?

In [5]:
sample = df.sample(frac=1, random_state=1)
sample.head()

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:
mask = np.random.rand(len(sample)) < 0.8
train = sample[mask].copy()
test = sample[~mask].copy()
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [7]:
print("Training set:\n ", train.Label.value_counts(normalize=True))
print("Test set:\n", test.Label.value_counts(normalize=True))

Training set:
  ham     0.866816
spam    0.133184
Name: Label, dtype: float64
Test set:
 ham     0.862352
spam    0.137648
Name: Label, dtype: float64


## The proportions are roughly the same as those from the complete dataset

# Letter Case and Punctuation

The next big step is to use the training set to teach the algorithm to classify new messages.

Recall from the previous mission that when a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations (note that we replaced "SpamC" with "Ham" inside the second equation below):

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Let's also summarize what the terms in the equations above mean:

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}
 
To calculate all these probabilities, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Right now, our training and test sets have this format (the messages are fictitious to make the example easier to understand):

<img src ="cpgp_dataset_1.png">

To make the calculations easier, we want bring the data to this format (the table below is a transformation of the table you see above):

<img src ="cpgp_dataset_2.png">
About the transformation above, notice that:

- The `SMS` column doesn't exist anymore.
- Instead, the `SMS` column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values `spam, 2, 2, 1, 1, 0, 0, 0, 0, 0`. These values tell us that:
  - The message is spam.
  - The word "secret" occurs two times inside the message.
  - The word "prize" occurs two times inside the message.
  - The word "claim" occurs one time inside the message.
  - The word "now" occurs one time inside the message.
  - The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

## Tasks
1. Remove all the punctuation from the `SMS` column. You can use the regex `'\W'` to detect any character that is not from a-z, A-Z or 0-9.
  - For instance, the function `re.sub('\W', ' ', 'Secret!! Money, goods.' )` strips the punctuation marks and outputs the string `'Secret Money goods '`.
  - For simplicity, you can use the `Series.str.replace()` method.
2. For each message, transform every letter in every word to lower case. You may want to use the `Series.str.lower()` method.

In [8]:
import re
sms = train.SMS.apply(lambda x: re.sub("\W", " ",x))
train["SMS"] = sms
train["SMS"] = train.SMS.str.lower()
train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,i forgot 2 ask ü all smth there s a card on ...
4,ham,ok i thk i got it then u wan me 2 come now or...


# Creating the vodabulary
On the previous screen, we removed the punctuation and changed all letters to lowercase. Recall that our end goal with this data cleaning process is to bring our training set to this format:
<img src ="cpgp_dataset_2.png">
With the exception of the "Label" column, every other column in the transformed table above represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message). Recall from the previous mission that we call the set of unique words a vocabulary.

We'll eventually bring the training set to that format ourselves, but first, let's create a list with all of the unique words that occur in the messages of our training set.

## Tasks
1. Create a vocabulary for the messages in the training set. The vocabulary should be a Python list containing all the unique words across all messages, where each word is represented as a string.
  - Begin by transforming each message from the `SMS` column into a list by splitting the string at the space character — use the `Series.str.split()` method.
  - Initiate an empty list named `vocabulary`.
  - Iterate over the the `SMS` column (each message in this column should be a list of strings by the time you start this loop).
    - Using a nested loop, iterate each message in the `SMS` column (each message should be a list of strings) and append each string (word) to the `vocabulary` list.
  - Transform the `vocabulary` list into a set using the `set()` function. This will remove the duplicates from the `vocabulary` list.
  - Transform the `vocabulary` set back into a list using the `list()` function.

In [9]:
vocabulary = []
train["SMS"] = train.SMS.str.split()

In [10]:
for message in train.SMS:
    for word in message:
        vocabulary.append(word)

In [11]:
vocabulary = set(vocabulary)
len(vocabulary)

7789

In [12]:
vocabulary = list(vocabulary)
len(vocabulary)

7789

# The Final Training Set
Now we're going to use the vocabulary to make the data transformation we need:

<img src = "cpgp_dataset_3.png">
Eventually, we're going to create a new DataFrame. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

For instance, to create the table we see above, we could use this dictionary and then convert it to a DataFrame:
```
word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }


word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()
```

|secret|prize|claim	|now|coming|to|my|party|winner|
|---|---|---|---|---|---|---|---|---|
|0	|2	|2	|1	|1	|0	|0	|0	|0	|0|
|1	|1	|0	|0	|0	|1	|1	|1	|1	|0|
|2	|1	|1	|1	|1	|0	|0	|0	|0	|1|

(As you may have noticed from the output above, the `Label` column is missing, but we'll get to that in the next exercise.)

To create the dictionary we need for our training set, we can use the code below, where:

- We start by initializing a dictionary named `word_counts_per_sms`, where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, where each element in the list is a `0`.
  - The code `[0] * 5` outputs `[0, 0, 0, 0, 0]`. So the code `[0] * len(training_set['SMS'])` outputs a list of the length of `training_set['SMS']`, where each element in the list will be a `0`.
- We loop over `training_set['SMS']` using at the same time the `enumerate()` function to get both the index and the SMS message (`index` and `sms`).
  - Using a nested loop, we loop over `sms` (where `sms` is a list of strings, where each string represents a word in a message).
     - We incremenent `word_counts_per_sms[word][index]` by `1`.
     
```
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
```
Now that we have the dictionary we need, let's do the final transformations to our training set and then move forward with creating the spam filter.

## Tasks
1. Run the code you see above to get the `word_counts_per_sms` dictionary. In case you want to do a bit of exploration, note that this is a large dictionary, and printing it all is not recommended (you should rather use a for loop and print only the first five or so key-value pairs).
2. Transform `word_counts_per_sms` into a DataFrame using `pd.DataFrame()`.
3. Concatenate the DataFrame we just built above with the DataFrame containing the training set (this way, we'll also have the `Label` and the `SMS` columns). Use the `pd.concat()` function.

In [13]:
word_counts_per_sms = {uniqe_word: [0]*len(train.SMS) for uniqe_word in vocabulary}
for index, sms in enumerate(train.SMS):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [14]:
word_counts = pd.DataFrame(word_counts_per_sms)

In [15]:
word_counts.head()

,pandy,videos,saves,morrow,04,nag,unemployed,vatian,tell,09056242159,...,42478,07742676969,copy,orh,plans,0808,wings,txts,browsin,okay
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
print("word_counts: ", word_counts.shape)
print("training_set: ", train.shape)

word_counts:  (4475, 7789)
training_set:  (4475, 2)


In [17]:
training_set = pd.concat([train, word_counts], axis=1)
training_set.shape

(4475, 7791)

In [18]:
training_set.head()

,Label,SMS,pandy,videos,saves,morrow,04,nag,unemployed,vatian,...,42478,07742676969,copy,orh,plans,0808,wings,txts,browsin,okay
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[ok, i, thk, i, got, it, then, u, wan, me, 2, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
training_set.isna().sum()

Label      0
SMS        0
pandy      0
videos     0
saves      0
          ..
0808       0
wings      0
txts       0
browsin    0
okay       0
Length: 7791, dtype: int64

# Calculating Constants first
Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. Recall that the Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary

Recall from the previous mission that:
- NSpam is equal to the number of words in all the spam messages — it's not equal to the number of spam messages, and it's not equal to the total number of unique words in spam messages.
- NHam is equal to the number of words in all the non-spam messages — it's not equal to the number of non-spam messages, and it's not equal to the total number of unique words in non-spam messages.

We'll also use Laplace smoothing and set \alpha = 1

## Tasks
Using the training set only:
1. Calculate P(Spam) and P(Ham). There's more than one way to write the code that can calculate this — feel free to choose any solution you want.
2. Calculate NSpam, NHam, NVocabulary. Feel free to choose any programming solution you like.
3. Initiate a variable named `alpha` with a value of `1`.

In [20]:
training_set.Label.value_counts(normalize=True)
p_ham = training_set.Label.value_counts(normalize=True).values[0]
p_spam = training_set.Label.value_counts(normalize=True).values[1]
print("Ham: ",p_ham)
print("Spam: ",p_spam)

Ham:  0.8668156424581006
Spam:  0.13318435754189945


In [21]:
NVocabulary = len(vocabulary)
training_set["Total_Words"] = training_set.sum(axis=1)

In [22]:
training_set[["Label","SMS","Total_Words"]].head()

,Label,SMS,Total_Words
0,ham,"[yep, by, the, pretty, sculpture]",5
1,ham,"[yes, princess, are, you, going, to, make, me,...",9
2,ham,"[welp, apparently, he, retired]",4
3,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",26
4,ham,"[ok, i, thk, i, got, it, then, u, wan, me, 2, ...",15


In [23]:
groups = training_set.groupby("Label")["Total_Words"].sum()
NSpam = groups.spam
NHam = groups.ham

In [24]:
print("NSpam: ",NSpam)
print("NHam: ",NHam)

NSpam:  15027
NHam:  57340


In [25]:
alpha = 1

# Calculating Parameters
As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(wi|Spam) and P(wi|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(wi|Spam) and P(wi|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

- "secret code"
- "secret party 2night"

We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

\begin{equation}
P(\text{"secret"}|Spam) = \frac{N_{"secret"|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

- P("lost"|Spam) and P("lost"|Ham)
- P("navigate"|Spam) and P("navigate"|Ham)
- P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(wi|Spam) and P(wi|Ham).

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called **parameters**.

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm will be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

Let's now calculate all the parameters using the equations below:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Recall that P(wi|Spam) and P(wi|Ham) are key parts of the equations that we need to classify the new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

## Tasks
1. Initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is `0`. We'll need one dictionary to store the parameters for P(wi|Spam), and the other for P(wi|Ham).
   - If the entire vocabulary were `['sea', 'navigate']`, we'd need to initialize two dictionaries, one for spam and one for ham, and both should look like this: `{'sea': 0, 'navigate': 0}`.
2. Isolate the spam and the ham messages in the training set into two different DataFrames. The `Label` column will help you isolate the messages.
3. Iterate over the vocabulary, and, for each word, calculate P(wi|Spam) and P(wi|Ham) using the formulas we mentioned above.
   - Recall that NSpam, NHam, NVocabulary, and `alpha` are already calculated from the last screen.
   - Recall from the previous mission that Nwi|Spam is equal to the number of times the word wi occurs in all the spam messages, while Nwi|Ham is equal to the number of times the word wi occurs in all the ham messages.
   - Once you're done with calculating an individual parameter, update the probability value in the two dictionaries you created initially.

In [29]:
p_w_spam = {unique_word: 0 for unique_word in vocabulary}
p_w_ham = {unique_word: 0 for unique_word in vocabulary}

In [40]:
spam = (training_set["Label"] == "spam")
ham = (training_set["Label"] == "ham")
spam_messages = training_set[spam].copy()
ham_messages = training_set[ham].copy()

In [44]:
ham_messages.shape

(3879, 7792)

In [45]:
spam_messages.shape

(596, 7792)

In [52]:
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam+alpha)/(NSpam+alpha)*NVocabulary
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham+alpha)/(NHam+alpha)*NVocabulary
    
    p_w_spam[word] = p_word_given_spam
    p_w_ham[word] = p_word_given_ham

# Classifying a New Message
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
  - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
  - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
  - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

Below, we see a rough sketch of how the spam filter function might look like:
```
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    '''    
    This is where we calculate:

    p_spam_given_message = ?
    p_ham_given_message = ?
    '''    

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')
```

For the `classify()` function above, note that:

- The input variable `message` is assumed to be a string.
- We perform a bit of data cleaning on the string `message`:
  - We remove the punctuation using the `re.sub()` function.
  - We bring all letters to lower case using the `str.lower()` method.
  - We split the string at the space character and transform it into a Python list using the `str.split()` method.
- There's some placeholder code for calculating `p_spam_given_message` and `p_ham_given_message` — we'll write this code in the exercise below.
- We compare `p_spam_given_message` with `p_ham_given_message` and then print a classification label.

To write the code we need for calculating `p_spam_given_message` and `p_ham_given_message`, we need to use these two equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Note that some new messages will contain words that are not part of the vocabulary. Recall from the previous mission that we simply ignore these words when we're calculating the probabilities.

Now we'll write the code for calculating `p_spam_given_message` and `p_ham_given_message`, and then we'll use the function to classify two new messages. On the next screen, we'll classify all the 1,114 messages in our test set.

## Tasks
1. Copy the `classify()` function you see above and write the code needed for calculating `p_spam_given_message` and `p_ham_given_message`.
- Initiate `p_spam_given_message` and `p_ham_given_message` with an initial value. We recommend initiating the variables as `p_spam_given_message = p_spam` and `p_ham_given_message = p_ham` (`p_spam` and `p_ham` are P(Spam) and P(Ham), and they were calculated on the previous steps).
- Iterate over each word in `message` (the input of the `classify()` function), which should be a list of strings by the time you start this loop. For each word:
  - If the word is present in the dictionary containing the spam parameters, then update the value of `p_spam_given_message` by multiplying with the parameter value specific to that word. You'll need to code something similar to `p_spam_given_message *= parameters_spam[word]`.
  - If the word is present in the dictionary containing the ham parameters, then update the value of `p_ham_given_message` by multiplying with the parameter value specific to that word. You'll need to do something like `p_ham_given_message *= parameters_spam[word]`.
  - If the word is not present in any of the two dictionaries, then don't do anything. Recall that we ignore words that are not part of the vocabulary.
2. Use the `classify()` function to classify two new messages. You can use any messages you want, but we suggest that one message is obviously spam, and the other is obviously ham. For instance, you can use these two messages:
- `'WINNER!! This is the secret code to unlock the money: C3421.`
- `"Sounds good, Tom, then see u there`

In [63]:
def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_w_spam:
            p_spam_given_message *= p_w_spam[word]
        if word in p_w_ham:
            p_ham_given_message *= p_w_ham[word]
        
    
    print("P(Spam|Message): ", p_spam_given_message)
    print("P(Ham|Message): ", p_ham_given_message)
    
    if p_spam_given_message > p_ham_given_message:
        return "spam"
    elif p_spam_given_message < p_ham_given_message:
        return "ham"
    else:
        return "Can´t decide! Let the human classify this!"
    

In [64]:
test_spam = "WINNER!! This is the secret code to unlock the money: C3421"
test_ham = "Sounds good, Tom, then see u there"
classify(test_spam)

P(Spam|Message):  286721390658.7536
P(Ham|Message):  675930934.5897093


'spam'

In [65]:
classify(test_ham)

P(Spam|Message):  10843.89754304565
P(Ham|Message):  17509383.52898659


'ham'

# Measuring the Spam Filter´s Accuracy
On the previous screen, we managed to create a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

First off, we'll change the `classify()` function that we wrote previously to return the labels instead of printing them. Below, note that we now have `return` statements instead of `print()` functions:

```
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
```
Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.
```
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()
```

|Label|	SMS|predicteLabel|	SMS	predicted|
|---|---|---|---|
|0	|ham|	Later i guess. I needa do mcat study too.|	ham|
|1	|ham|	But i haf enuff space got like 4 mb...|	ham
|2	|spam|	Had your mobile 10 mths? Update to latest Oran...	|spam|
|3	|ham|	All sounds good. Fingers . Makes it difficult ...	|ham|
|4	|ham|	All done, all handed in. Don't know if mega sh...	|ham|

Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

## Tasks
1. Measure the accuracy of the spam filter.
- Initialize a variable named `correct` with a value of `0`.
- Initialize a variable named `total` with the number of messages in the test set.
- Iterate over the test set DataFrame (you can use the `DataFrame.iterrows()` method). For each row:
  - If the actual label is the same as the predicted label, then increment `correct` by `1`.
- Use `correct` and `total` in combination with the above formula to calculate the accuracy of the spam filter.
2. What do you think about the accuracy value? Is it better or worse than you expected?

In [69]:
#test["predicted"] = test["SMS"].apply(classify)
test.head()

,Label,SMS,predicted
0,ham,Havent.,ham
1,ham,My uncles in Atlanta. Wish you guys a great se...,ham
2,ham,Ok which your another number,ham
3,spam,FreeMsg Why haven't you replied to my text? I'...,spam
4,ham,Ooooooh I forgot to tell u I can get on yovill...,ham


In [70]:
correct = 0
total = len(test.Label)

In [84]:
for row in test.iterrows():
    row = row[1]
    if row.Label == row.predicted:
        correct +=1

In [85]:
correct

1079

In [86]:
accuracy = correct/total

In [87]:
accuracy

0.98359161349134

In [88]:
wrong = total-correct

In [89]:
wrong

18

# Next Steps
In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

If you want to keep working on this project, here's a few next steps you can take:

- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.
- Get the project portfolio-ready by using a few tips from our style guide for data science projects.